In [ ]:
#!/usr/bin/env python
# coding: utf-8
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean
import math
from math import sqrt
import os
import scipy.stats as stats
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
get_ipython().run_line_magic('matplotlib', 'inline')
# read table from excel, create two dataframe, one for test data, the other for practice data

In [ ]:
dftest = pd.read_excel (r'/Users/xiaokangdong/Desktop/tableforcast.xlsx', sheet_name='test')
dfpra = pd.read_excel (r'/Users/xiaokangdong/Desktop/tableforcast.xlsx', sheet_name='pra')
# create time list and demand list on test data
tlist = dftest['t'].tolist()
print(tlist)
xtlist = dftest['xt'].tolist()
print(xtlist)
# create time list and demand list on practice data
tplist = dfpra['t'].tolist()
xtplist = dfpra['xt'].tolist()
print(xtplist)
# plot demand against time to see the trend
plt.plot(tlist, xtlist, 'ro')
plt.xlabel('time')
plt.ylabel('Demand')
plt.show()
# plot demand against time to see the trend
plt.plot(tlist, xtplist, 'ro')
plt.xlabel('time')
plt.ylabel('Demand')
plt.show()
# Smoothing average function to generate demand on N
alist = []
def SMA(N):
for t in tlist[N-1:]:
at = (sum(xtlist[t-(N-1):t]))/N
alist.append(at)
print(alist)
SMA(10)
N = 10
alpha = 2/(N+1)
# Exponential smoothing function to generate demand
def ES_a(alpha,a0,xlist):
a1 = a0
alist = [a1]
for t in range(1,len(tlist)):
at = round((alpha * xlist[t]+ (1- alpha) * alist[t-1]),5)
alist.append(at)
return alist
# Exponential smoothing function to generate MAD
def ES_MAD(alpha,a0,MAD0,xlist):
a1 = a0
MAD1 = round((1-alpha)*MAD0,6)
alist = [a1]
MADlist = [MAD1]
for t in range(1,len(tlist)):
at = round((alpha * xlist[t]+ (1- alpha) * alist[t-1]),5)
MADt = round((alpha * abs(alist[t-1]-xlist[t])+ (1- alpha) * MADlist[t-1]),6)
alist.append(at)
MADlist.append(MADt)
return MADlist
# EOQ function to obtain Q
def EOQ(list,A,H):
Qlist = []
for t in range(0,len(tlist)):
Qt = sqrt(2* A * list[t] / H* 12)
Qlist.append(round(Qt))
return Qlist
print(EOQ(al,300,3))
# generate Gamma function
def G(x):
G = stats.norm.pdf(x, loc = 0, scale = 1)- x * (1- stats.norm.cdf(x, loc = 0, scale = 1))
return(G)
# generate mu prime on leadtime
def mu(ltime, a):
mulist = [i * ltime for i in a]
return(mulist)
# generate sig prime on leadtime
def sigma(ltime, MAD):
sigmalist = [math.sqrt(math.pi /2) * math.sqrt(ltime) * i for i in MAD]
return(sigmalist)
# generate S2 function
def S2(mu, sigma, Q, R):
S = 1 - (sigma/ Q)* (G((R - mu)/ sigma) - G((R + Q - mu)/ sigma))
return(S)
# forecast demand list, MAD list, Q list, mu,sigma based on test data
al = ES_a(2/11,20,xtlist)
MADl = ES_MAD(2/11,20,5,xtlist)
Ql = EOQ(al,300,3)
mul = mu(7,ES_a(2/11,20,xtlist))
sigl = sigma(7,ES_MAD(2/11,20,5,xtlist))
# forcast demand list, MAD list, Q list, mu,sigma based on practice data
al_p = ES_a(2/13,56,xtplist)
MADl_p = ES_MAD(2/13,56,14,xtplist)
Ql_p = EOQ(al_p,290,3)

mul_p = mu(5,al_p)
sigl_p = sigma(5,MADl_p)


# function to generate R accordingly
R = []
for i in range(0,30):
    def Reorder_fun(mu,sig,Q,level):
            for r in range(0,Q[i]):
                ss = S2(mu[i], sig[i], Q[i], r)
                if ss >= level:
                    return r
                    
    
    reorder = Reorder_fun(mul,sigl,Ql,0.98) 
    R.append(reorder)

print(R)

R_p = []
for i in range(0,30):
    def Reorder_fun(mu,sig,Q,level):
            for r in range(0,Q[i]):
                ss = S2(mu[i], sig[i], Q[i], r)
                if ss >= level:
                    return r
                
    reorder = Reorder_fun(mul_p,sigl_p,Ql_p,0.9530)
    R_p.append(reorder)
    
print(R_p)


# dataframe for t, x, a, MAD, Q, R based on test data

data = pd.DataFrame([tlist,xtlist,al,MADl,Ql,R]) #Each list would be added as a row
data = data.transpose() #To Transpose and make each rows as columns
data.columns=['t','x','a','MAD','Q','R'] #Rename the columns
data[['t','x','Q','R']]= data[['t','x','Q','R']].astype('int')
print(data.head())

# dataframe for t, x, a, MAD, Q, R based on practice data

data = pd.DataFrame([tlist,xtplist,al_p,MADl_p,Ql_p,R_p]) #Each list would be added as a row
data = data.transpose() #To Transpose and make each rows as columns
data.columns=['t','x','a','MAD','Q','R'] #Rename the columns
data[['t','x','Q','R']]= data[['t','x','Q','R']].astype('int')
print(data.head())

data.to_csv(r'/Users/xiaokangdong/Desktop/Book1.csv')


# get H function
def Hfun(x):
    d = (1/2) * ((x * x + 1)*(1 - stats.norm.cdf(x))- x * stats.norm.pdf(x))
    return d

# get Q prime function and R prime function based on 6.14 and 5.52
 

i = 0 
def Qnext(Q,R):
    Rmu = (R- mul[i])/ sigl[i]
    RQmu = (R + Q - mul[i])/ sigl[i]
    Qnext = math.sqrt((2*300* al[i])/3*12 + 2 *(sigl[i]**2)*(Hfun(Rmu)- Hfun(RQmu)- (Q / sigl[i])*G(RQmu)))
    return int(Qnext+1)

print(Qnext(220,143))
Qnew = Qnext(220,143)


def RR(mu,sig,Q,level):
    for r in range(0,Q):
        ss = S2(mu, sig, Q, r)
        if ss >= level:
            return r
                    
    
print(RR(mul[i],sigl[i],219,0.98))

Rnew = RR(mul[i],sigl[i],Qnew,0.98)



# do iteration for Q and R until Q doesn’t change on practice data
list = []
for i in range(0,30):

    def RQ(Q):
        while True:
            R = RR(mul[i],sigl[i],Q,0.98)
            Qnew = Qnext(Q,R)
            if Qnew == Q:
                return (Qnew,R)
                break
            else:
                Q = Qnew
        
    list.append(RQ(Ql[i]))
    
print(list)


# get Q prime function and R prime function based on 6.14 and 5.52

i = 0 
def Qnext_p(Q,R):
    Rmu = (R- mul_p[i])/ sigl_p[i]
    RQmu = (R + Q - mul_p[i])/ sigl_p[i]
    Qnext = math.sqrt((2*290* al_p[i])/3*12 + 2 *(sigl_p[i]**2)*(Hfun(Rmu)- Hfun(RQmu)- (Q / sigl_p[i])*G(RQmu)))
    return int(Qnext+1)

Qnew_p = Qnext_p(360,274)

print(Qnext_p(360,274))
 
def RR_p(mu,sig,Q,level):
    for r in range(0,Q):
        ss = S2(mu, sig, Q, r)
        if ss >= level:
            return r
                    
    
print(RR_p(mul_p[i],sigl_p[i],360,0.9530))

Rnew_p = RR_p(mul_p[i],sigl_p[i],Qnew_p,0.9530)


# do iteration for Q and R until Q doesn’t change on practice data

list_p = []
for i in range(0,30):

    def RQ_p(Q):
        while True:
            R = RR_p(mul_p[i],sigl_p[i],Q,0.9530)
            Qnew = Qnext_p(Q,R)
            if Qnew == Q:
                return (Qnew,R)
                break
            else:
                Q = Qnew
                continue
        
    list_p.append(RQ_p(Ql_p[i]))
    
print(list_p)


d = pd.DataFrame(list_p) #list would be transferred to dataframe
d.columns=['Qprime','Rprime'] #Rename the columns
d[['Qprime','Rprime']]= d[['Qprime','Rprime']].astype('int')
print(d.head())

d.to_csv(r'/Users/xiaokangdong/Desktop/Book2.csv')